In [1]:
import itertools
from pathlib import Path
from pprint import pprint

from matplotlib import pyplot as plt, cm
import numpy as np
import pandas as pd
#from pandas_path import path
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import minmax_scale
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from keras.preprocessing import sequence
from tqdm import tqdm

#pd.set_option("max_colwidth", 80)
RANDOM_SEED = 711  # For reproducibility

ModuleNotFoundError: No module named 'tensorflow.keras'

#### parameters

In [ ]:
DATA_PATH = "C:/Users/Seth Johnston/Documents/DrivenData/Mars/01_Data"

#### Create a function to organize a data file

In [ ]:
metadata = pd.read_csv(DATA_PATH + "/metadata.csv")
print(metadata.shape)
metadata.head(3)

In [ ]:
test_file = metadata[(metadata['split'] == 'train') &
                    (metadata['instrument_type'] == 'commercial')]['features_path'][25]
print(test_file)
temp_df = pd.read_csv(DATA_PATH + "/train_features/" + test_file)
print(temp_df.shape)
temp_df.head(2)

In [ ]:
temp_df['m/z'].describe()

In [ ]:
temp_df['time'].hist()

In [ ]:
# first, get time delta values to use in each step
ls_time = sorted(list(set(temp_df['time'])))
len(ls_time)

In [ ]:
def time_delta(ls):
    sorted_ls = sorted(list(set(ls)))
    ret_ls = []
    for i,t in enumerate(sorted_ls):
        if i==0:
            ret_ls.append(t)
        else:
            ret_ls.append(t - sorted_ls[i-1])
    return ret_ls

test_diff = time_delta(temp_df['time'])
print(len(test_diff))
test_diff[290:]

In [ ]:
m_df = temp_df[(temp_df['m/z'] == 2)].copy()
print(m_df.shape)
m_df.head(2)

In [ ]:
m_ts = time_delta(m_df['time'])
print(len(m_ts))
m_ts[294:]

In [ ]:
m_df['time_delta'] = m_ts
m_array = np.array(m_df[['time_delta', 'temp', 'abundance']])
m_array.shape

In [ ]:
def create_array(mz, df, trim_size=290):
    try:
        m_df = df[(df['m/z'] == mz)].copy()
        m_ts = time_delta(m_df['time'])
        m_df['time_delta'] = m_ts
        m_array = np.array(m_df[['time_delta', 'temp', 'abundance']])
        print(m_array.shape)
        m_pad = np.pad(m_array, (trim_size,3))
        print(m_pad.shape)
        return m_pad
    except:
        return np.zeros((trim_size, 3))
    


In [ ]:
temp_df.head(2)

In [ ]:
create_array(2, temp_df)